# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob, types
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, ScalarFormatter, FuncFormatter
from matplotlib.patches import ConnectionPatch, FancyArrowPatch
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run Lesion_Size.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','MotorPreserved.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

---

String Format for Scientific Notation

In [ ]:
def SciNote(string):
    """
    Format numbers with Real scientific notation
    Ex: 'p-val={}'.format(SciNote(p))
    """
    f = ScalarFormatter(useOffset=False, useMathText=True)
    g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
    fmt = FuncFormatter(g)
    return fmt(string)

---

plotting group bar plots

In [ ]:
def plot_event_rec(root, ax, Profiles, badAnimals=None, TaskParamToPlot='% good trials', shift=0,
                   nPre=5, nPost=10, **kwarg):

    if badAnimals is None:
        badAnimals=[]
    
    #getting the data
    animalList,SessionDict=event_detect(root, Profiles[0], Profiles[1], badAnimals=badAnimals)
    SessionDict={animal:SessionDict[animal] for animal in animalList}
    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                      SessionDict,
                                                      parameter=param,
                                                      redo=False,
                                                      TaskParamToPlot=[TaskParamToPlot])

    assert nPre<=nSessionPre and nPost<=nSessionPost,"fewer sessions available than requested:"
    
    data=np.array(list(Results[TaskParamToPlot].values()))
    
    xData=np.append(np.arange(-nPre,0),np.arange(1,nPost+1))

    groupData=np.nanmedian(data,axis=0)
    
    groupErr=np.nanpercentile(data,(25,75),axis=0)
    
    #plotting the errorbar
    ax.errorbar(xData + shift, groupData[nSessionPre-nPre:nSessionPre+nPost],
                abs(groupErr-groupData)[:,nSessionPre-nPre:nSessionPre+nPost],
                fmt='-o', zorder=5, ms=2, elinewidth=1,alpha=.8, **kwarg)
    
    def _tik(x,pos):
        if x in [-nPre,-1,1,nPost]:
            return ('%+g' if x>0 else '%g')%x
        else:
            return ''
    ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(_tik))
    
    ax.set_xlim([-nPre-1,nPost+1])
    ax.spines['bottom'].set_bounds(-nPre,nPost)
    ax.set_xticks(xData)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylim([0,100])
    ax.set_ylabel(TaskParamToPlot)
    ax.set_xlabel('Sessions relative lesion')

    return data[:,nSessionPre]


def add_legend_for_lesion_types(ax,N):
    r_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='xkcd:red', marker='o',
                                       markerfacecolor='xkcd:red', markersize=3, label=f'DLS\n$n={N[1]}$')
    b_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='xkcd:blue', marker='o',
                                       markerfacecolor='xkcd:blue', markersize=3, label=f'DMS\n$n={N[2]}$')
    m_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='xkcd:magenta', marker='o',
                                       markerfacecolor='xkcd:magenta', markersize=3, label=f'DS\n$n={N[3]}$')
    k_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='gray', marker='o',
                                       markerfacecolor='gray', markersize=3, label=f'Control\n$n={N[0]}$')

#     errorbar = matplotlib.lines.Line2D([0,0], [0,1], color='k', label='95% CI')


    leg=ax.legend(handles=[k_marker,r_marker,b_marker,m_marker],loc=(0,.8),mode='expand',ncol=4,
                  facecolor=None,edgecolor=None, fontsize='xx-small',frameon=False)
    return leg

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'10',
              'Speed':'10',
              'Tag':['Control','Control-BackTo10','Control-Late-BackTo10']
              }
    profile2={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'10',
              'Speed':'10',
              'Tag':['Control-AfterBreak']
              }

    #number of sessions to plot
    badAnimals=['RatBAD']
    TaskParamToPlot="% good trials"
    wspace=0.05
    
    
    Profiles=(profile1,profile2,)
    plt.close('all')
    fig=plt.figure(figsize=(5,4))
    ax=fig.add_subplot(111)

    a=plot_event_rec(root, ax, Profiles,color='k',shift=-.1*.25)
    ax.set_ylim([0,100])

---

Plot the speed definition graphic

In [ ]:
def plot_speed_def(ax):
    x0=np.arange(0,7,.01)
    y0=10*x0+10

    x1=np.arange(7, 8.16,.01)
    y1=-60*x1+500

    x=np.append(x0,x1)
    y=np.append(y0,y1)

    y=smooth(y*1.1,sigma=50,mode='constant')

    x40=np.where(np.logical_and(y<=40,x>7))[0][0]
    x60=np.where(np.logical_and(y<=60,x>7))[0][0]
    #plot the traj
    ax.plot(x, y, lw=2, c='navy', )
    #plot the 40 and 60 horizontal lines
    ax.hlines(y=40, xmin=-10, xmax=x[x40], lw=ax.spines['bottom'].get_lw(), ls='--')
    ax.hlines(y=60, xmin=-10, xmax=x[x60],lw=ax.spines['bottom'].get_lw(), ls='--')
    #plot the red piece
    ax.plot([x[x40],x[x60]],[40,60], lw=ax.spines['bottom'].get_lw(), c='r',zorder=4)
    #plot the x-axis at the red piece
    ax.arrow(x[x60],60,1.3,0,lw=ax.spines['bottom'].get_lw(),head_width=2,head_length=.2, ec='r', fc='r',zorder=5)
    # plot the curve
    x2=7.5
    y2=y[np.where(x>=x2)[0][0]]
    ax.add_patch(FancyArrowPatch(posA=(x[x60]+.8,62), posB=(x2-.1,y2-2),
                                 arrowstyle='-', connectionstyle='arc3,rad=-.5',color='r',
                                 mutation_scale=15,lw=ax.spines['bottom'].get_lw(),zorder=6))
    #writing 
    ax.text(7.9,40,'speed',rotation=-75, c='r', fontsize='xx-small',ha='left',va='center')
    
    
    ax.set_xticks([0,1,2,3,4,5,6,7,8])
    ax.set_yticks(range(0,91,10))
    ax.set_yticklabels([0,'',20,'',40,'',60,'',80,''])
    ax.spines['bottom'].set_bounds(0,8)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_bounds(0,90)
    ax.set_xlim([-.1,9])
    ax.set_ylim([-2,90])
    ax.set_ylabel('Position (cm)')
    ax.set_xlabel('Trial time (s)')


In [ ]:
if "__file__" not in dir():

    plt.close('all')
    fig=plt.figure(figsize=(5,5))
    ax=fig.add_subplot(111)
    plot_speed_def(ax)

---

plotting group double errorbars for speed 

In [ ]:
def plot_event_1on1(root, ax, Profiles, colorCode, badAnimals=None, TaskParamToPlot='% good trials',
                   x_pos=None, nPre=slice(-3,None), nPost=slice(0,3),nFin=slice(3,6),
                    seed=1, animal_plot=True):
    if badAnimals is None:
        badAnimals=[]
    
    if x_pos is None:
        diff=.35
        x_c=ax.get_xlim()[1]
        x_pos=(x_c-diff,x_c+diff,x_c+3*diff)
    diff=x_pos[1]-x_pos[0]
    
    #getting the data
    animalList,sessionDict=event_detect(root, Profiles[0], Profiles[1], badAnimals=badAnimals)
    
    data=np.empty((len(animalList),3))
    for i,animal in enumerate(animalList):
        preSession,postSession=sessionDict[animal][0], sessionDict[animal][1]
        out=animal_learning_stats(root, animal, PerfParam=TaskParamToPlot,
                                  goodSessions=[*preSession,*postSession],redo=False)
        
        data[i,0]=np.nanmedian(out[:len(preSession)][nPre])
        data[i,1]=np.nanmedian(out[len(preSession):][nPost])
        data[i,2]=np.nanmedian(out[len(preSession):][nFin])
        
            
    
    y=np.nanpercentile(data,50,axis=0)
    yerr=np.nanpercentile(data,(25,75),axis=0)
    
    #plotting the errorbar
    ax.errorbar(x_pos, y, abs(yerr-y),fmt='o', zorder=5, ms=2, elinewidth=1, color='k')
    
    #plotting individual animals
    if animal_plot:
        np.random.seed(seed=seed)
        _coeff=5
        for i in range(data.shape[0]):
            if np.any(np.isnan(data[i,:])):
                continue
            jitPre =np.random.uniform(low=x_pos[0]-diff/_coeff, high=x_pos[0]+diff/_coeff, size=1)
            jitPost=np.random.uniform(low=x_pos[1]-diff/_coeff, high=x_pos[1]+diff/_coeff, size=1)
            jitFin =np.random.uniform(low=x_pos[2]-diff/_coeff, high=x_pos[2]+diff/_coeff, size=1)
            
            _,tag=lesion_type(root,animalList[i])
            c=colorCode[tag] if tag in colorCode else 'gray'
            ax.scatter([jitPre,jitPost,jitFin],data[i,:], s=1, c=c, marker='o', edgecolors='none', alpha=.8)
            ax.plot([jitPre,jitPost,jitFin],data[i,:], c=c, lw=.2, alpha=.4)
#             ax.plot([jitPre,jitFin],data[i,[0,-1]], c=c, lw=.2, alpha=.4)

    ax.set_xlim([x_pos[0]-diff/2,x_pos[-1]+diff/2])
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Before','After','Final'])
    ax.xaxis.set_tick_params(rotation=0)
    ax.spines['bottom'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylabel(TaskParamToPlot)
    ax.set_xlabel('Sessions relative to lesion')

    return data,animalList

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout',
              'Control-NoTimeout-Control','Control-Sharp','IncReward-Late-Sharp',
              'Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
             }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Late-Lesion_DLS','Late-Lesion_DMS','Late-Lesion_DS',
                    'Late-Lesion_DS-Sharp','Late-Lesion_DMS-Sharp']
             }

    #number of sessions to plot
    badAnimals=['RatBAD']
    TaskParamToPlot="% good trials"
    wspace=0.05
    
    
    Profiles=(profile1,profile2,)
    plt.close('all')
    fig=plt.figure(figsize=(5,4))
    ax=fig.add_subplot(111)

    ax.set_xticks([])
    plot_event_1on1(root, ax, Profiles, colorCode={'Control':'r'}, badAnimals=None, TaskParamToPlot=TaskParamToPlot,
                       x_pos=None, nPre=slice(-3,None), nPost=slice(0,3),nFin=slice(3,6),
                        seed=1, animal_plot=True)

---

plot the speed correlation plot

In [ ]:
def late_lesion_correlation_with_size(root, ax, Profiles, badAnimals, color, TaskParamToPlot:str,
                                      preSlice=slice(-5,None), postSlice=slice(0,5)):
    
    _,sessionDict=event_detect(root, Profiles[0], Profiles[1], badAnimals=badAnimals)
    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                 sessionDict,
                                                 parameter=param,
                                                 redo=False,
                                                 TaskParamToPlot=[TaskParamToPlot])

    data=np.array(list(Results[TaskParamToPlot].values()))
    slicedPreData  = np.ones((data.shape))*np.nan
    slicedPostData = np.ones((data.shape))*np.nan
    
    for row,Dnan in enumerate(data):
        Dpre= Dnan[:nSessionPre][~ np.isnan(Dnan[:nSessionPre])]
        Dpost=Dnan[nSessionPre:][~ np.isnan(Dnan[nSessionPre:])]
        preData =Dpre[preSlice]
        postData=Dpost[postSlice]

        slicedPreData[row,:len(preData)]=preData
        slicedPostData[row,:len(postData)]=postData
    
    yPre =np.nanmean(slicedPreData ,axis=1)
    yPost=np.nanmean(slicedPostData,axis=1)

    size=[]
    behav=[]
    animals=[]
    for i,animal in enumerate(Results[TaskParamToPlot].keys()):
        if np.isnan(yPre[i]) or np.isnan(yPost[i]):
            logging.error(f'{animal}: {TaskParamToPlot} not defined!')
            continue
        try:
            Hist=HistologyExcel('/NAS02',animal)
        except Exception as e:
            logging.error(f'{animal}: {repr(e)}')
            continue
        
        behav.append(yPost[i]-yPre[i])
        size.append(Hist.lesion_size())
        animals.append(animal)
        
        #plotting
        _,tag=lesion_type(root,animal)
        ax.scatter(size[-1], behav[-1], s=4, c=color[tag], label=tag);
#         plt.annotate(animal[-3:], (size[-1],behav[-1]), fontsize='xx-small')
    
    ax.set_xticks(np.arange(0,1.01,.1))
    ax.set_xticklabels(['0','','','','','0.5','','','','','1'])
    ax.spines['bottom'].set_bounds(0,1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlim([-.02,1.02])
    ax.set_ylabel(f'$\Delta$ {TaskParamToPlot}')
    ax.set_xlabel('Lesion size')

    
    return behav, size, animals

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout',
              'Control-NoTimeout-Control','Control-Sharp','IncReward-Late-Sharp',
              'Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
             }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Late-Lesion_DLS','Late-Lesion_DMS','Late-Lesion_DS',
                    'Late-Lesion_DS-Sharp','Late-Lesion_DMS-Sharp']
             }

    Profiles=(profile1,profile2)
    TaskParamToPlot="Forward Running Speed"
    preSlice=slice(-5,None)
    postSlice=slice(3,8)
    
    color={'DS':'r','DMS':'g', 'DLS':'b', 'Control':'k'}
    
    plt.close('all')
    ax=plt.subplot(111)

    
    a=late_lesion_correlation_with_size(root, ax=ax, Profiles=Profiles, badAnimals=[],color=color, TaskParamToPlot=TaskParamToPlot,
                                      preSlice=preSlice, postSlice=postSlice)

---

plot speed vs max pos trial-by-trial

In [ ]:
def maxPos_speed_correlation(data):
    _MinX_=30
    _,maxTrials,maxPos=sequentialTrials(data, xFront=_MinX_)._compute_max_pos()
    spd=forwardRunningSpeed(data,minXBack=_MinX_+10,minXFront=_MinX_).compute()
    
    MaxTrialSpd={trial:spd[trial] for trial in maxTrials if trial in spd.keys()}
    MaxTrialPos=np.array([maxPos[i] for i,trial in enumerate(maxTrials) if trial in MaxTrialSpd.keys()])
    MaxTrialSpd=np.array(list(MaxTrialSpd.values()))
        
    return MaxTrialPos,MaxTrialSpd

def plot_animal_maxPos_speed_relation(root, ax, profile, colorCode, Slice=slice(-5,None)):
    
    animalList=batch_get_animal_list(root, profile)
    
    pos={}
    spd={}
    for animal in animalList:
        data=data_fetch(root, animal, profile, PerfParam=[maxPos_speed_correlation], NbSession=Slice)['maxPos_speed_correlation']
        
        pos[animal]=np.array([i for sessionData in data for i in sessionData[0]])
        spd[animal]=np.array([i for sessionData in data for i in sessionData[1]])        
        
        
        
    return pos,spd

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout',
              'Control-NoTimeout-Control','Control-Sharp','IncReward-Late-Sharp',
              'Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
             }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Late-Lesion_DLS','Late-Lesion_DMS','Late-Lesion_DS',
                    'Late-Lesion_DS-Sharp','Late-Lesion_DMS-Sharp']
             }

    Profiles=(profile1,profile2)
    TaskParamToPlot="Forward Running Speed"
    preSlice=slice(-5,None)
    postSlice=slice(3,8)
    
    color={'DS':'r','DMS':'g', 'DLS':'b', 'Control':'k'}
    
    plt.close('all')
    ax=plt.subplot(111)

    a=plot_animal_maxPos_speed_relation(root, ax, profile1, color, Slice=slice(-5,None))

In [ ]:
x1,y1=plot_animal_maxPos_speed_relation(root, ax, profile1, color, Slice=slice(-5,None))
x2,y2=plot_animal_maxPos_speed_relation(root, ax, profile2, color, Slice=slice(-5,None))

In [ ]:
import matplotlib.backends.backend_pdf


In [ ]:
pdf = matplotlib.backends.backend_pdf.PdfPages("/home/david/Documents/output.pdf")
animalList=batch_get_animal_list(root, profile2)
for rat in x2:
    fig=plt.figure()
    ax1=plt.subplot(121)
    ax1.plot(x1[rat],y1[rat],'.')
    ax1.set_title(rat)
    ax1.set_xlim([0,90])
    ax1.set_ylim([0,90])
    ax1.set_xlabel('max pos')
    ax1.set_ylabel('speed')
    
    ax2=plt.subplot(122)
    ax2.plot(x2[rat],y2[rat],'.')
    try:
        ax2.set_title(lesion_type(root,rat)[1]+ str(HistologyExcel('/NAS02',rat).lesion_size()).format('%0.2f'))
    except:pass
    ax2.set_xlim([0,90])
    ax2.set_ylim([0,90])
    
    pdf.savefig( fig )

clear_output()
pdf.close()

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    CtrlColor='gray'
    DLSColor='xkcd:red'
    DMSColor='xkcd:blue'
    DSColor='xkcd:magenta'
    
    ColorCode={'DS':DSColor,
               'DMS':DMSColor,
               'DLS':DLSColor,
               'Control':CtrlColor
              }
    
    BadLateRats=('Rat223','Rat231')
       

    
    #===============================================
    
    # GRID 1 PARAMS
    
    _s=.1
    ShiftDLS1 =-1*_s
    ShiftDMS1 =0*_s
    ShiftDS1  =1*_s
    
    nPre1=5
    nPost1=10
    
    TaskParamToPlot1="Forward Running Speed"
    
    profilePreLesion1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout', 'Control-NoTimeout-Control',
                     'Control-Sharp','IncReward-Late-Sharp','Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
              }
    
    profileDLS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DLS']
                 }
    profileDMS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DMS','Late-Lesion_DMS-Sharp'],
                 }  
    profileDS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DS','Late-Lesion_DS-Sharp']
                 }  

    ProfilesDLS1=(profilePreLesion1,profileDLS1,)
    ProfilesDMS1=(profilePreLesion1,profileDMS1,)
    ProfilesDS1 =(profilePreLesion1,profileDS1,)
    
    
    #================================================
    
    # GRID 3 PARAMS

    profileLesions3={'Type':'Good',
                     'rewardType':'Progressive',
                     'option':['not used', 'AsymmetricLesion'],
                     'initialSpeed':['0','10'],
                     'Speed':'10',
                     'Tag': list(set((*profileDLS1['Tag'], *profileDMS1['Tag'], *profileDS1['Tag'])))
                     }

    Profiles3=(profilePreLesion1,profileLesions3)
    
    preSlice3=slice(-3,None)
    postSlice3=slice(0,3)
    finSlice3=slice(7,10)

Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(6,1.2)
    fig=plt.figure(figsize=figsize,dpi=600)
    
        

    ##########################################
    # 1: % speed time course
    gs1= fig.add_gridspec(nrows=1, ncols=1, left=0, bottom=0, right=0.47, top=1)
    ax1= fig.add_subplot(gs1[0])
    
    #DLS
    plot_event_rec(root, ax1, ProfilesDLS1,TaskParamToPlot=TaskParamToPlot1,
                   nPre=nPre1, nPost=nPost1,color=DLSColor,shift=ShiftDLS1)

    #DMS
    plot_event_rec(root, ax1, ProfilesDMS1,TaskParamToPlot=TaskParamToPlot1,
                   nPre=nPre1, nPost=nPost1,color=DMSColor,shift=ShiftDMS1)

    #DS
    plot_event_rec(root, ax1, ProfilesDS1,TaskParamToPlot=TaskParamToPlot1,
                   nPre=nPre1, nPost=nPost1,color=DSColor,shift=ShiftDS1)

    
    ax1.set_ylim([20,100])
    ax1.spines['left'].set_bounds(20,100)
    ax1.set_yticks(range(20,101,20))
    ax1.set_ylabel('Speed (cm/s)')


    

    ##########################################
    # 2: speed event plot
    gs2= fig.add_gridspec(nrows=1, ncols=1, left=0.47, bottom=0, right=0.7, top=1)
    ax2= fig.add_subplot(gs2[0])
    
    plot_event_1on1(root, ax2, Profiles3, colorCode=ColorCode, TaskParamToPlot=TaskParamToPlot1,
                    nPre=preSlice3, nPost=postSlice3,nFin=finSlice3)

    ax2.set_ylim([20,100])
    ax2.spines['left'].set_bounds(20,100)
    ax2.set_yticks(range(20,101,20))
    ax2.set_yticklabels([])
    ax2.set_ylabel('')
    ax2.set_xlabel('')
    
    
    ##########################################
    # 3: % speed correlation
    gs3= fig.add_gridspec(nrows=1, ncols=1, left=0.77, bottom=0, right=1, top=1)
    ax3= fig.add_subplot(gs3[0])
  
    behav3, size3, animals3=late_lesion_correlation_with_size(root, ax=ax3, Profiles=Profiles3, badAnimals=[],
                                                              color=ColorCode,
                                                              TaskParamToPlot=TaskParamToPlot1,
                                                              preSlice=preSlice3, postSlice=postSlice3)
    
    r3,p3=stats.pearsonr(size3, behav3)
    s=f'$n={len(animals3)}$ rats'+'\n'+'$r=$'+'$%+.2f$'%(r3)+'\n'+'$p=$'+'{}'.format(SciNote(p3))
    ax3.text(1,10,s, ha='right',va='top', fontsize='xx-small')
    
    ax3.set_ylim([-52,12])
    ax3.spines['left'].set_bounds(-50,10)
    ax3.set_yticks([-50,-40,-30,-20,-10,0,10])
    ax3.set_yticklabels([-50,'',-30,'',-10,'',10])
    ax3.set_ylabel('$\Delta$Speed (cm/s)')
    
    


    
#     fig.align_ylabels([axes1[0],axes2[0],axes3[0],axes4[0]])
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#     AXES=(axes4[0],ax1,ax2,ax5,ax7,ax8,ax6)
#     OFFX=np.array([.07]*len(AXES))
#     OFFY=np.array([.01]*len(AXES))
#     OFFX[5]=0.03
#     OFFX[[0,1,2,4,6]]=0.05
    
#     add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','MotorPreserved.pdf'),
                format='pdf', bbox_inches='tight')
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()